In [1]:
!pip install efficientnet_pytorch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from sklearn import metrics

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=9f0165b9a692acc9f945284200d771879480fadf6367add55e39d9211c84302f
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class Dataset(Dataset):
    def __init__(self, root_images, root_file, transform = None):
        self.root_images= root_images
        self.transform  = transform
        self.file       = pd.read_csv(root_file)

    def __len__(self):
        return self.file.shape[0]
    
    def __getitem__(self, index):
        img_path     = os.path.join(self.root_images, self.file['filename'][index])
        image        = np.array(Image.open(img_path).convert('RGB'))

        if self.transform is not None:
            augmentations = self.transform(image=image)
            image         = augmentations['image']

        return image

In [3]:
LR    = 1e-4
BS    = 16
NE    = 100
H     = 256+20
W     = 300+20

num_tta    = 6
IMG_Train  = '/kaggle/input/cgiar-eyes-on-the-ground-challenge/content/content/train'
IMG_Test   = '/kaggle/input/cgiar-eyes-on-the-ground-challenge/content/content/test'

FILE_Train = '/kaggle/input/cgiar-eyes-on-the-ground-challenge/train_folds.csv'
FILE_Test  = '/kaggle/input/cgiar-eyes-on-the-ground-challenge/Test.csv'

In [4]:
train = pd.read_csv(FILE_Train)
test = pd.read_csv(FILE_Test)
SampleSubmission = pd.read_csv('/kaggle/input/cgiar-eyes-on-the-ground-challenge/SampleSubmission.csv')

In [5]:
def get_loaders(image_test, image_train, file_test, file_train, bs, transform):

    test_ds         = Dataset(root_images=image_test, root_file= file_test, transform=transform)
    test_loader     = DataLoader(test_ds, batch_size = bs, shuffle=False)

    return  test_loader

In [6]:
def tta_augmentations(W, H):
    tta_transform = A.Compose(
        [
            A.Resize(width=W, height=H),
            A.HorizontalFlip(p=0.5),
            A.Rotate(limit=30),
            A.RandomCrop(height=H-20, width=W-20, p=1),
            normalize,
            ToTensorV2()
        ],
    )
    return tta_transform

normalize = A.Normalize(
    mean=[0.5, 0.5, 0.5],
    std=[0.5, 0.5, 0.5],
    max_pixel_value= 255.0
)

In [7]:
def tta_predictions(model, loader, num_tta=4):
    model.eval()
    k = 0
    predictions     = []
    predictions_tmp = []
    
    with torch.no_grad():
        for _ in range(num_tta):
            for x in tqdm(loader):
                x = x.to('cuda').to(torch.float32)
                p = model(x)
                p = p.cpu().detach().numpy()
                predictions_tmp.append(p)
            
            predictions_tmp = np.vstack(predictions_tmp)
            predictions.append(predictions_tmp)
            predictions_tmp = []
    # Calculate the average of predictions
    avg_predictions = np.mean(predictions, axis=0)
    return avg_predictions

In [8]:
def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [9]:
Test               = pd.read_csv('/kaggle/input/cgiar-eyes-on-the-ground-challenge/Test.csv')
Test['extent']  = 0

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.modelA = EfficientNet.from_pretrained("efficientnet-b5")
        self.fc1 = nn.Linear(1000, 1)

        self.relu = nn.ReLU()
        
    def forward(self, img):
        x = self.modelA(img)
        x = self.fc1(x)
        return self.relu(x)

In [11]:
transform = tta_augmentations(W, H)
for fold in range(1):
    print(' --------------------------------------------------- Fold: ' +str(fold))
    model     = Net().to('cuda')
    optimizer = optim.Adam(model.parameters(), lr = LR)
    CHECKPOINT_FILE = '/kaggle/input/icgiar-eyes-on-the-ground-train-v1-eff-b3/baseline_0.pth.tar'
    
    test_loader = get_loaders(
        IMG_Test, IMG_Train, FILE_Test, FILE_Train, BS, transform
    )
    load_checkpoint(torch.load(CHECKPOINT_FILE), model, optimizer)
    
    avg_predictions = tta_predictions(model, test_loader, num_tta)

    k = 0
    avg_predictions = avg_predictions.flatten()
    for i in range(len(avg_predictions)):
        Test['extent'][k] = avg_predictions[i]
        k += 1

 --------------------------------------------------- Fold: 0


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth
100%|██████████| 117M/117M [00:00<00:00, 301MB/s] 


Loaded pretrained weights for efficientnet-b5
=> Loading checkpoint


100%|██████████| 542/542 [03:50<00:00,  2.35it/s]
/tmp/ipykernel_27/142964871.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test['extent'][k] = avg_predictions[i]
/tmp/ipykernel_27/142964871.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test['extent'][k] = avg_predictions[i]
/tmp/ipykernel_27/142964871.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test['extent'][k] = avg_predictions[i]
/tmp/ipykernel_27/142964871.p

In [12]:
Test = Test[['ID', 'extent']]

In [13]:
Test.to_csv('Test_extent.csv', index=False)
